In [1]:
############ USE ONLY IF YOU HAVE ROS ############

import sys
sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages') # in order to import cv2 under python3

##################################################

import numpy as np
import pandas as pd
import cv2

colnames = ['filename', 'file_size', 'file_attributes', 'region_count', 'region_id', 'region_shape_attributes', 'region_attributes']
data = pd.read_csv('data/via_export_csv.csv', names=colnames)

In [2]:
name = data.filename.tolist()
region_shape = data.region_shape_attributes.tolist()

In [3]:
def keypoints_parsing(image, reg_shape, begin, num):

    image = cv2.imread("data/"+str(image))
    
    output = []
    keypoints = [None]*14
    dim = [None]*2
    cone = 0
    
    if image is None:
        print("File does not exist")
    
    else:
        
        for kp in range(num):
            for i in range(7):
                string = reg_shape[begin+cone].split(',')

                for j in range(2):
                    temp = string[j+1].split(':')

                    if j==1:
                        temp[1] = temp[1].replace("}", "")

                    dim[j] = int(temp[1])

                x, y = dim[0], dim[1]
                
                keypoints[2*i] = int(x)
                keypoints[2*i+1] = int(y)

                cone += 1
                
            output.append(keypoints[:])
            
        return output

In [4]:
def bboxes_parsing(image, reg_shape, begin, num):
    
    image = cv2.imread("data/"+str(image))
    
    output = []
    dim = [None]*4
    img_count = 0
    
    if image is None:
        print("File does not exist")

    else:        
        for boxes in range(num):
            string = reg_shape[begin+img_count].split(',')
            for j in range(4):
                temp = string[j+1].split(':')

                if j==3:
                    temp[1] = temp[1].replace("}", "")

                dim[j] = int(temp[1])
            
            output.append(dim[:])
            img_count += 1

        return output

In [5]:
def get_range(reg_shape,begin,ann_type):
    
    num = 0
    
    if ann_type == 'keypoints':
        
        while (begin+num) < len(reg_shape) and len(reg_shape[begin+num]) < 37:
            num += 1
        return (num)/7
    
    elif ann_type == 'bboxes':
        
        while (begin+num) < len(reg_shape) and len(reg_shape[begin+num]) > 37:
            num += 1        
        return num

In [56]:
kp_num = []
bb_num = []

temp = np.array([])
temp1 = []
current = 1
no_kp = [None]*14
kp = []
bb = []
Y_train = []

In [57]:
while current<len(name):
    
    k = get_range(region_shape, int(current), 'keypoints')
    keypoints = keypoints_parsing(name[int(current)], region_shape, int(current), int(k))
    kp.append(keypoints[:])
    current += k*7
    
    b = get_range(region_shape, int(current), 'bboxes')
    bboxes = bboxes_parsing(name[int(current)], region_shape, int(current), int(b))
    bb.append(bboxes[:])
    current += b
    
    for i in range(b):
        if i<k:
            temp = np.append(bboxes[i],keypoints[i])

        else:
            temp = np.append(bboxes[i],no_kp)
        
        temp1.append(temp)
    
    Y_train.append(temp1)

In [58]:
Y_train

[[array([648, 445,  46,  70, 672, 453, 662, 471, 661, 488, 658, 502, 681,
         471, 684, 487, 686, 502]),
  array([694, 390,  34,  49, 711, 400, 705, 410, 704, 421, 702, 431, 717,
         410, 717, 420, 719, 431]),
  array([1274,  441,   54,   61, 1299,  445, 1292,  461, 1291,  476, 1286,
          490, 1308,  462, 1309,  478, 1314,  491]),
  array([1061,  386,   35,   36, 1080,  391, 1076,  401, 1074,  408, 1073,
          418, 1084,  402, 1085,  410, 1088,  419]),
  array([930, 371, 24, 22, None, None, None, None, None, None, None, None,
         None, None, None, None, None, None], dtype=object),
  array([834, 358, 16, 17, None, None, None, None, None, None, None, None,
         None, None, None, None, None, None], dtype=object),
  array([665, 376, 24, 26, None, None, None, None, None, None, None, None,
         None, None, None, None, None, None], dtype=object),
  array([622, 364, 20, 20, None, None, None, None, None, None, None, None,
         None, None, None, None, None, No

In [59]:
len(Y_train)

51